# 关于Keras模型
Keras有两种类型的模型，序贯模型（Sequential）和函数式模型（Model），函数式模型应用更为广泛，序贯模型是函数式模型的一种特殊情况。

两类模型有一些方法是相同的：
- `model.summary()`：打印出模型概况，它实际调用的是`keras.utils.print_summary`
- `model.get_config()`:返回包含模型配置信息的Python字典。模型也可以从它的`config`信息中重构回去

```python
config = model.get_config()
model = Model.from_config(config)
# or, for Sequential:
model = Sequential.from_config(config)
```

- `model.get_layer()`：依据层名或下标获得层对象
- `model.get_weights()`：返回模型权重张量的列表，类型为`numpy.array`
- `model.set_weights()`：从 `numpy.array` 里将权重载入给模型，要求数组具有与 `model.get_weights()` 相同的形状。
- `model.to_json`：返回代表模型的 JSON 字符串，仅包含网络结构，不包含权值。可以从 JSON 字符串中重构原模型：

```python
from models import model_from_json

json_string = model.to_json()
model = model_from_json(json_string)
```

- `model.to_yaml`：与 `model.to_json` 类似，同样可以从产生的 YAML 字符串中重构模型

```python
from models import model_from_yaml

yaml_string = model.to_yaml()
model = model_from_yaml(yaml_string)
```

- `model.save_weights(filepath)`：将模型权重保存到指定路径，文件类型是 HDF5（后缀是`.h5`）
- `model.load_weights(filepath, by_name=False)`：从 HDF5 文件中加载权重到当前模型中, 默认情况下模型的结构将保持不变。如果想将权重载入不同的模型（有些层相同）中，则设置 `by_name=True`，只有名字匹配的层才会载入权重。

## 函数式（Functional）模型
函数式模型称作Functional，但它的类名是 `Model`，因此我们有时候也用Model来代表函数式模型。

Keras函数式模型接口是用户定义多输出模型、非循环有向模型或具有共享层的模型等复杂模型的途径。一句话，只要你的模型不是类似VGG一样一条路走到黑的模型，或者你的模型需要多于一个的输出，那么你总应该选择函数式模型。函数式模型是最广泛的一类模型，序贯模型（Sequential）只是它的一种特殊情况。更多关于序列模型的资料参考： [序贯模型API](http://keras-cn.readthedocs.io/en/latest/models/sequential/#sequential)

# 1 第一个模型：全连接网络
在开始前，有几个概念需要澄清：
- 层对象接受张量为参数，返回一个张量。
- 输入是张量，输出也是张量的一个框架就是一个模型，通过 `Model` 定义。
- 这样的模型可以被像Keras的 `Sequential` 一样被训练

```python
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training
```

## 所有的模型都是可调用的，就像层一样
利用函数式模型的接口，我们可以很容易的重用已经训练好的模型：你可以把模型当作一个层一样，通过提供一个 `tensor` 来调用它。注意当你调用一个模型时，你不仅仅重用了它的结构，也重用了它的权重。

In [7]:
x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)

这种方式可以允许你快速的创建能处理序列信号的模型，你可以很快将一个图像分类的模型变为一个对视频分类的模型，只需要一行代码：

In [8]:
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)

## 多输入和多输出模型
使用函数式模型的一个典型场景是搭建多输入、多输出的模型。

In [10]:
from keras.layers import Conv2D, MaxPooling2D, Input

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

更多内容参考：[快速开始函数式（Functional）模型](http://keras-cn.readthedocs.io/en/latest/getting_started/functional_API/)

### 示例

In [14]:
from keras.models import Model
from keras.layers import Input, Dense

In [26]:
# 定义输入层 Input，主要是为了定义输入的矩阵尺寸
a = Input(shape= (None, 28, 28))

# 定义各个连接层
x = Dense(64, activation= 'relu')(a)
x = Dense(64, activation= 'relu')(x)

# 定义输出层
y = Dense(10, activation= 'relu')(x)

In [19]:
# 定义模型对象
model = Model(inputs= a, outputs= y)

In [29]:
# 编译模型

model.compile(optimizer= 'rmsprop', loss= 'categorical_crossentropy', metrics= ['accuracy'])
